In [ ]:
from ortools.sat.python import cp_model
#https://developers.google.com/optimization/reference/python/sat/python/cp_model
import pandas as pd
import plotly.express as px

In [ ]:
### Variables ###

In [ ]:
days = range(5) ###Eventually model will be extended to a month or year
sessions = range(17)
tracers = range(4)

###TODO: Add variable for second PET scanner and staff schedules; Design matrix?
#Human Readable
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
sess_times = ["08:00 - 10:00","08:30 - 10:30","09:00 - 11:00",
             "09:30 - 11:30","10:00 - 12:00","10:30 - 12:30",
             "11:00 - 13:00","11:30 - 13:30","12:00 - 14:00",
             "12:30 - 14:30","13:00 - 15:00","13:30 - 15:30",
             "14:00 - 16:00","14:30 - 16:30","15:00 - 17:00",
             "15:30 - 17:30","16:00 - 18:00"]
tracers_names = ["PIB","AV1451","MK6240","UCB-J"]

In [ ]:
### No use for design matrix yet but could be used to 
#weight individual sessions
first = [1,1,1,1,1,
         1,1,1,1,1,
         1,1,1,1,1,
         1,1]

second = [first, first, first, first, first]
third = [second, second, second, second]
#print(third)

In [ ]:
### Creates the model CP Model: 
#https://developers.google.com/optimization/reference/python/sat/python/cp_model
model = cp_model.CpModel()

In [ ]:
### Decision Variables ###

In [ ]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is a Bool of whether day/session/tracer is
#instance is scheduled or not
x = {}
for t in tracers:
    for d in days:
        for s in sessions:
            x[(t,d,s)] = model.NewBoolVar(
                "{}{}{}".format(tracers_names[t],
                                day_names[d],
                                sess_times[s]))

In [ ]:
#Decision Variables to link outcomes/concurrent end states

In [ ]:
#Allows us to sum instances when a PIB scan is followed by an MK6240 scan in the same day
#Consider removing t1 and t2 since do not change and not necessary; more readable though?
t1 = tracers_names.index("PIB")
t2 = tracers_names.index("MK6240")
pib_mk_sess_by_day = {}
for s in sessions:
    for t in [t1,t2]:
        for d  in days:
            pib_mk_sess_by_day[(t,s,d)] = model.NewBoolVar("{}{}{}".format(tracers_names[t],
                                                                           day_names[d],
                                                                           sess_times[s]))
pib_mk_concur = {}             
for s in range(len(sessions)-4): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+4,len(sessions)): #MK6240 scan must start after end of PIB
        pib_mk_concur[(t1,s,t2,sess)] = model.NewBoolVar("{}{};{}{}".format(tracers_names[t1],
                                                                         sess_times[s],
                                                                         tracers_names[t2],
                                                                         sess_times[sess]))
pib_mk_same_day = {}
for s in range(len(sessions)-4): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+4,len(sessions)): #MK6240 scan must start after end of PIB
        for d in days:
            pib_mk_same_day[(t1,s,t2,sess,d)] = model.NewBoolVar("{}{};{}{};{}".format(tracers_names[t1],
                                                                                       sess_times[s],
                                                                                       tracers_names[t2],
                                                                                       sess_times[sess],
                                                                                       day_names[d]))

In [ ]:
#Allows us to sum instances when there are multiple MK6240 scans in a day
mk = tracers_names.index("MK6240")
mk_sess_by_day = {}
for s in sessions:
    for d in days:
            mk_sess_by_day[(mk,s,d)] = model.NewBoolVar("{}{}{}".format(tracers_names[mk],
                                                                        day_names[d],
                                                                        sess_times[s]))
mk_concur = {}             
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+1,len(sessions)): #MK6240 scan must start after end of PIB
        mk_concur[(mk,s,sess)] = model.NewBoolVar("{};{}{}".format(tracers_names[mk],
                                                                         sess_times[s],
                                                                         sess_times[sess]))
mk_same_day = {}
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+1,len(sessions)): #MK6240 scan must start after end of PIB
        for d in days:
            mk_same_day[(mk,s,sess,d)] = model.NewBoolVar("{};{}{};{}".format(tracers_names[mk],
                                                                                       sess_times[s],
                                                                                       sess_times[sess],
                                                                                       day_names[d]))

In [ ]:
#Allows us to sum instances when there are multiple AV1451 scans in a day
av = tracers_names.index("AV1451")
av_sess_by_day = {}
for s in sessions:
    for d in days:
            av_sess_by_day[(av,s,d)] = model.NewBoolVar("{}{}{}".format(tracers_names[av],
                            day_names[d],
                            sess_times[s]))
av_concur = {}             
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an AV1451 scan
    for sess in range(s+1,len(sessions)): #AV1451 scan must start after first AV1451
        av_concur[(av,s,sess)] = model.NewBoolVar("{};{}{}".format(tracers_names[av],
                                                                         sess_times[s],
                                                                         sess_times[sess]))
av_same_day = {}
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an AV1451 scan
    for sess in range(s+1,len(sessions)): #MAV1451 scan must start after first AV1451
        for d in days:
            av_same_day[(av,s,sess,d)] = model.NewBoolVar("{};{}{};{}".format(tracers_names[av],
                                                                                       sess_times[s],
                                                                                       sess_times[sess],
                                                                                       day_names[d]))

In [ ]:
#Allows us to sum instances when there are multiple PIB scans in a day
pib = tracers_names.index("PIB")
pib_sess_by_day = {}
for s in sessions:
    for d in days:
            pib_sess_by_day[(pib,s,d)] = model.NewBoolVar("{}{}{}".format(tracers_names[pib],
                            day_names[d],
                            sess_times[s]))
pib_concur = {}             
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an PIB scan
    for sess in range(s+1,len(sessions)): #PIB scan must start after first PIB
        pib_concur[(pib,s,sess)] = model.NewBoolVar("{};{}{}".format(tracers_names[pib],
                                                                         sess_times[s],
                                                                         sess_times[sess]))
pib_same_day = {}
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an PIB scan
    for sess in range(s+1,len(sessions)): #PIB scan must start after first PIB
        for d in days:
            pib_same_day[(pib,s,sess,d)] = model.NewBoolVar("{};{}{};{}".format(tracers_names[pib],
                                                                                       sess_times[s],
                                                                                       sess_times[sess],
                                                                                       day_names[d]))

In [ ]:
### Hard Constraints ###

In [ ]:
#Constraints creating distinct sessions

In [ ]:
#Each session gets at most 1 tracer
for d in days:
    for s in sessions:
        model.Add(sum(x[(t,d,s)] for t in tracers) <= 1)

In [ ]:
#2 hr slots without overlap
for d in days:
    for s in sessions[0:14]:
        overlap = []
        for t in tracers:
            overlap.extend([
                x[(t,d,s)],x[(t,d,s+1)],
                x[(t,d,s+2)],x[(t,d,s+3)]
            ])
        model.Add(sum(overlap) <= 1)

In [ ]:
#Each tracer gets at least 1 session per week
for t in tracers:
    all_sess = []
    for d in days:
        for s in sessions:
            all_sess.append(x[(t,d,s)])
    model.Add(sum(all_sess) >= 1)

In [ ]:
#PIB constraints

In [ ]:
#PIB 3 max per day/batch
t = tracers_names.index("PIB")
for d in days[1:4]:
    p_day = []
    for s in sessions:
        p_day.append(x[(t,d,s)])
        model.Add(sum(p_day) <= 3)

In [ ]:
#PIB 1 max per Monday
t = tracers_names.index("PIB")
d = day_names.index("Mon")
p_mon = []
for s in sessions:
    p_day.append(x[(t,d,s)])
model.Add(sum(p_day) <= 1)

In [ ]:
#PIB 3 hr gap between PIB scans
t = tracers_names.index("PIB")
for d in days:
    for s in sessions[0:12]:
        p_gap = []
        for num in range(6):
            p_gap.append(x[(t,d,s+num)])
        model.Add(sum(p_gap) <= 1)

In [ ]:
#AV1451 constraints

In [ ]:
#12 per month for AV1451; Avgerage to 3 per week for weekly model
t = tracers_names.index("AV1451")
a_week = []
for d in days:
    for s in sessions:
        a_week.append(x[(t,d,s)])
model.Add(sum(a_week) <= 3)

In [ ]:
#24HR between production but 2 scans per production batch/day 
t = tracers_names.index("AV1451")

for d in days:
    a_2 = []
    for s in sessions:
        a_2.append(x[(t,d,s)])
    model.Add(sum(a_2) <= 2)

In [ ]:
#12:00 earliest sess for AV1451
t = tracers_names.index("AV1451")
a_early = []
for d in days:
    for s in sessions[0:8]:
        a_early.append(x[(t,d,s)])
        #print("x{}{}{}".format(d,s,t)) #sum() = 0
model.Add(sum(a_early) == 0)

In [ ]:
#MK6240 constraints

In [ ]:
#MK6240 max 3 sessions each day
t = tracers_names.index("MK6240")
for d in days:
    m_3 = []
    for s in sessions:
        m_3.append(x[(t,d,s)])
    model.Add(sum(m_3) <= 3)

In [ ]:
#MK6240 only sessions 10 through 16; Tues-Fri
t = tracers_names.index("MK6240")
m_days = []
#sess_not = [0,1,2,3,4,5,6,7,8,16]
for d in days[1:5]:
    for s in list(sessions[0:9])+list(sessions[16:18]):
        m_days.append(x[(t,d,s)])
model.Add(sum(m_days) == 0)

In [ ]:
#MK6240 only sessions 8 through 13; Mon
t = tracers_names.index("MK6240")
d = day_names.index("Mon")
m_mon = []
for s in list(sessions[0:7])+list(sessions[13:18]):
    m_mon.append(x[(t,d,s)])
model.Add(sum(m_mon) == 0)

In [ ]:
#UCB-J constraints

In [ ]:
#UCB-J max 1 session each day
t = tracers_names.index("UCB-J")
for d in days:
    u_1 = []
    for s in sessions:
        u_1.append(x[(t,d,s)])
    model.Add(sum(u_1) <= 1)

In [ ]:
#UCB-J session 6 or 12 Tues-Fri
t = tracers_names.index("UCB-J")
for d in days[1:5]:
    u_sess = []
    for s in list(sessions[0:5])+list(sessions[6:11])+list(sessions[12:18]):
        u_sess.append(x[(t,d,s)])
    model.Add(sum(u_sess) == 0)

In [ ]:
#UCB-J session 14 Mon
t = tracers_names.index("UCB-J")
d = day_names.index("Mon")
u_mon = []
for s in list(sessions[0:13])+list(sessions[14:18]):
    u_mon.append(x[(t,d,s)])
model.Add(sum(u_mon) == 0)

In [ ]:
#UCB-J 90-min gap prior to session during which other 11C tracers cannot be scanned [PIB,ER176,UCB-J]
#UCB-J constraint already included with 1 session per day constraint
#ER176 is not considered in this model
t = tracers_names.index("UCB-J")
t2 = tracers_names.index("PIB")
for d in days[1:5]:
    u_sess = []
    for s in [sessions[5],sessions[11]]: #only sessions 6 and 12 on these days
        #3 sessions before these sessions must be not have scans for [PIB,ER176,UCB-J]
        u_sess.append(x[(t,d,s)])
        u_sess.append(x[(t2,d,s-1)])
        u_sess.append(x[(t2,d,s-2)])
        u_sess.append(x[(t2,d,s-3)])
    model.Add(sum(u_sess) == 0)

In [ ]:
#Constraints for decision variables that link outcomes/concurrent end states

In [ ]:
#Incentivize having PIB scan followed by MK6240 in same day
t1 = tracers_names.index("PIB")
t2 = tracers_names.index("MK6240")
#Link pib_mk_concur with pib_mk_sess_by_day
for s in range(len(sessions)-4): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+4,len(sessions)): #MK6240 scan must start after end of PIB
        for d in days:
            #Link pib_mk_same_day with pib_mk_sess_by_day. Keeps variables in sync
            pib_day = pib_mk_sess_by_day[(t1,s,d)]
            mk_day = pib_mk_sess_by_day[(t2,s,d)]
            same_day_pairing = pib_mk_same_day[(t1,s,t2,sess,d)]
            model.AddBoolOr([pib_day.Not(), mk_day.Not(), same_day_pairing])
            # if same_day_pairing is True, then pib_day and mk_day must be True
            model.AddImplication(same_day_pairing, pib_day)
            model.AddImplication(same_day_pairing, mk_day)

        #Link pib_mk_concur with pib_mk_same_day
        pib_then_mk = sum(pib_mk_same_day[((t1,s,t2,sess,d))] for d in days)
        model.Add(pib_then_mk == pib_mk_concur[(t1,s,t2,sess)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having multiple MK6240
mk = tracers_names.index("MK6240")
#Link mk_concur with mk_sess_by_day
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an MK6240 scan
    for sess in range(s+1,len(sessions)): #MK6240 scan must start after end of first MK6240
        for d in days:
            #Link mk_same_day with mk_sess_by_day. Keeps variables in sync
            #for t in [t1,t2]:
            mk_1_day = mk_sess_by_day[(mk,s,d)]
            mk_2_day = mk_sess_by_day[(mk,sess,d)]
            same_day_pairing = mk_same_day[(mk,s,sess,d)]
            model.AddBoolOr([mk_1_day.Not(), mk_2_day.Not(), same_day_pairing])
            # if same_day_pairing is True, then mk_1_day and mk_2_day must be True
            model.AddImplication(same_day_pairing, mk_1_day)
            model.AddImplication(same_day_pairing, mk_2_day)

        #Link mk_concur with mk_same_day
        two_mk = sum(mk_same_day[((mk,s,sess,d))] for d in days)
        model.Add(two_mk == mk_concur[(mk,s,sess)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having multiple AV1451
av = tracers_names.index("AV1451")
#Link av_concur with av_sess_by_day
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an AV1451 scan
    for sess in range(s+1,len(sessions)): #AV1451 scan must start after end of first AV1451
        for d in days:
            #Link mk_same_day with mk_sess_by_day. Keeps variables in sync
            #for t in [t1,t2]:
            av_1_day = av_sess_by_day[(av,s,d)]
            av_2_day = av_sess_by_day[(av,sess,d)]
            same_day_pairing = av_same_day[(av,s,sess,d)]
            model.AddBoolOr([av_1_day.Not(), av_2_day.Not(), same_day_pairing])
            # if same_day_pairing is True, then av_1_day and av_2_day must be True
            model.AddImplication(same_day_pairing, av_1_day)
            model.AddImplication(same_day_pairing, av_2_day)

        #Link mk_concur with mk_same_day
        two_av = sum(av_same_day[((av,s,sess,d))] for d in days)
        model.Add(two_av == av_concur[(av,s,sess)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
#Incentivize having multiple PIB
PIB = tracers_names.index("PIB")
#Link av_concur with pib_sess_by_day
for s in range(len(sessions)-1): #avoid sessions that could not be followed by an PIB scan
    for sess in range(s+1,len(sessions)): #PIB scan must start after end of first PIB
        for d in days:
            #Link pib_same_day with pib_sess_by_day. Keeps variables in sync
            #for t in [t1,t2]:
            pib_1_day = pib_sess_by_day[(pib,s,d)]
            pib_2_day = pib_sess_by_day[(pib,sess,d)]
            same_day_pairing = pib_same_day[(pib,s,sess,d)]
            model.AddBoolOr([pib_1_day.Not(), pib_2_day.Not(), same_day_pairing])
            # if same_day_pairing is True, then pib_1_day and pib_2_day must be True
            model.AddImplication(same_day_pairing, pib_1_day)
            model.AddImplication(same_day_pairing, pib_2_day)

        #Link pib_concur with pib_same_day
        two_pib = sum(pib_same_day[((pib,s,sess,d))] for d in days)
        model.Add(two_pib == pib_concur[(pib,s,sess)])

#https://dougfenstermacher.com/blog/combinatorial-optimization#5-task-assignment

In [ ]:
### Objective ###

In [ ]:
# pylint: disable=g-complex-comprehension
pib = tracers_names.index("PIB")
mk = tracers_names.index("MK6240")
av = tracers_names.index("AV1451")
model.Maximize(
    sum(x[(t, d, s)] for t in tracers for d in days for s in sessions)
    + sum(3 * pib_mk_concur[pib,s,mk,sess] for s in range(len(sessions)-4) for sess in range(s+4,len(sessions)))
    + sum(3 * mk_concur[mk,s,sess] for s in range(len(sessions)-1) for sess in range(s+1,len(sessions)))
    + sum(2 * av_concur[av,s,sess] for s in range(len(sessions)-1) for sess in range(s+1,len(sessions)))
    + sum(3 * pib_concur[pib,s,sess] for s in range(len(sessions)-1) for sess in range(s+1,len(sessions)))
)
#First sum incentivizes scheduling any scans
#Second sum incentivizes PIB followed by an MK
#Third sum incentivizes multiple MK6240 sessions in one day
#Fourth sum incentivizes multiple AV1451 sessions in one day
#Fifth sum incentivizes multiple PIB sessions in one day
#Coefficients are arbitrary weights

In [ ]:
### Instanciate Solver, solve, and output end states ###

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)


if status == cp_model.OPTIMAL:
    print('Solution:')
    results = []
    for d in days:
        print(day_names[d])
        day = ["Day{}".format(d)]
        for t in tracers:
            for s in sessions:
                if solver.Value(x[(t, d, s)]) == 1:
                    day.append([d,t,s,1])
                    if third[t][d][s] == 1:
                        #weighted
                        print(tracers_names[t], 'scheduled', sess_times[s])
                    else:
                        print(tracers_names[t], 'scheduled', sess_times[s],
                              '(not requested).')
        print()
        results.append(day)
    pd.DataFrame(results).to_csv("schedule_maximize_solution.csv",index=False)e
    #Prints CSV file of solution
else:
    print('No optimal solution found !')

In [ ]:
### Create Gantt Graph of Solution ###

In [ ]:
sch_data = pd.DataFrame([
    dict(Session=sess_times[0], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[1], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[2], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[3], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[4], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[5], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[6], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[7], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[8], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[9], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[10], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[11], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[12], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[13], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[14], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[15], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[16], Start=0, Finish=5, Tracer="NA", color="1")
])

#Graph solution
#Read solution from CSV and then add scheduled sessions to Gantt
sch = pd.read_csv("schedule_maximize_solution.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
tracers_colors = ["2","3","4","5"]
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[3]) == 1:
                sch_entry = dict(Session=sess_times[int(ent_l[2])],
                                Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                Tracer=tracers_names[int(ent_l[1])], 
                                 color=tracers_colors[int(ent_l[1])])
                sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()
fig.show()

In [ ]:
#fig.to_dict() #use if want to adjust formatting

In [ ]:
#Save Gantt Graph as HTML
file_name = 'scheduling_max_graph'
fig.write_html(f"{file_name}.html")